In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import make_column_transformer

In [2]:
#EDA
data = pd.read_csv('Q2.csv')
data.shape
data.info()
data.describe()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hours Studied                     10000 non-null  int64  
 1   Previous Scores                   10000 non-null  int64  
 2   Extracurricular Activities        10000 non-null  object 
 3   Sleep Hours                       10000 non-null  int64  
 4   Sample Question Papers Practiced  10000 non-null  int64  
 5   Performance Index                 10000 non-null  float64
dtypes: float64(1), int64(4), object(1)
memory usage: 468.9+ KB


,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,Yes,9,1,91.0
1,4,82,No,4,2,65.0
2,8,51,Yes,7,2,45.0
3,5,52,Yes,5,2,36.0
4,7,75,No,8,5,66.0


In [3]:
#To count the number of null values in each features
data.isnull().sum()

Hours Studied                       0
Previous Scores                     0
Extracurricular Activities          0
Sleep Hours                         0
Sample Question Papers Practiced    0
Performance Index                   0
dtype: int64

In [4]:
#normalize
data.loc[(data['Extracurricular Activities']=="Yes"),['Extracurricular Activities']]=1
data.loc[(data['Extracurricular Activities']=="No"),['Extracurricular Activities']]=0
data=data.dropna()

scaler=MinMaxScaler()
normalized_df=scaler.fit_transform(data)
normalized_data= pd.DataFrame(normalized_df,columns=data.columns)
normalized_data.head(10)


,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,0.750,1.000000,1.0,1.0,0.111111,0.900000
1,0.375,0.711864,0.0,0.0,0.222222,0.611111
2,0.875,0.186441,1.0,0.6,0.222222,0.388889
3,0.500,0.203390,1.0,0.2,0.222222,0.288889
4,0.750,0.593220,0.0,0.8,0.555556,0.622222
5,0.250,0.644068,0.0,1.0,0.666667,0.566667
6,0.750,0.559322,1.0,0.2,0.666667,0.588889
7,0.875,0.084746,1.0,0.0,0.666667,0.355556
8,0.500,0.627119,0.0,0.8,0.222222,0.566667
9,0.375,0.830508,0.0,0.0,0.000000,0.655556


In [5]:
features=normalized_data.drop('Performance Index',axis=1)
target=normalized_data['Performance Index']
#split data to 3 parts(train(70%)-test(15%)-dev(15%))
X_train1,X_test,Y_train1,Y_test= train_test_split(features,target,test_size=0.15,random_state=2)
X_train,X_dev,Y_train,Y_dev= train_test_split(features,target,test_size=0.15,random_state=2)
print(features.shape,X_train.shape,X_test.shape,X_dev.shape)

(10000, 5) (8500, 5) (1500, 5) (1500, 5)


In [6]:
#importing tensorflow module
model=tf.keras.Sequential([tf.keras.layers.InputLayer(input_shape=5),
                           tf.keras.layers.Dense(15,activation=tf.keras.activations.relu),
                           tf.keras.layers.Dense(25,activation=tf.keras.activations.relu),
                           tf.keras.layers.Dense(1)
                           ])

In [7]:
def MSE(y_true,y_predicted):
  loss= tf.keras.losses.MSE(y_true,y_predicted)
  return loss

In [8]:
model.compile(
    loss=MSE,
    optimizer=Adam(),
    metrics=[MSE]
)

In [9]:
model.fit(X_train,Y_train,epochs=100,verbose=0)

In [10]:
preds=model.predict(X_test)

47/47 [==============================] - 0s 1ms/step


In [11]:
print('R_ score is : ',r2_score(Y_test,preds))

R_ score is :  0.9881148217055618


In [12]:
print('Mean Absolute Error : ',mean_absolute_error(Y_test,preds))

Mean Absolute Error :  0.018494517785310742


In [ ]:
# fitting the size of the plot
plt.figure(figsize=(16, 8))
# plotting training and test
plt.plot([i for i in range(len(Y_test))],Y_test, label="actual values", c='r')
plt.plot([i for i in range(len(Y_test))],preds, label="Predicted values", c='g')
# showing the plotting
plt.legend()
plt.show()

در اصل معیاری می باشد برای اینکه مدل ما چقدر خوب توانسته است داده ها را پیش بینی کند واز میانگین مربع خروجی واقعی و خروجی مورد انتظار بدست می اید در اصل یک لاست فانکشن می باشد برای مسایل رگرسیون.mse
برای کاهش لاست فانکشن در هنگام ترین کردن شبکه عصبی به کار می رودAdam